In [15]:
import os
import numpy as np
import pandas as pd
#statistical testing libraries
from scipy import stats
from scipy.stats import chi2_contingency

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


In [16]:
# Paths (must match Script 01 output structure)
OUTDIR = "output"
DATA_PATH = os.path.join(OUTDIR, "shipping_comments_clean.csv")
# Statistical testing libraries
TAB_DIR = os.path.join(OUTDIR, "tables")
os.makedirs(TAB_DIR, exist_ok=True)

print("Using data:", DATA_PATH)
print("Saving tables to:", TAB_DIR)


Using data: output/shipping_comments_clean.csv
Saving tables to: output/tables


In [17]:
#Load data
df = pd.read_csv(DATA_PATH)

print("Loaded shape:", df.shape)
print("Columns:", df.columns.tolist())
# Ensure required columns exist
required_cols = ["company", "text_clean"]
missing = [c for c in required_cols if c not in df.columns]

if missing:
    raise ValueError(f"Missing required columns from Script 01: {missing}")


Loaded shape: (2006, 14)
Columns: ['company', 'subreddit', 'post_id', 'comment_id', 'comment_created_utc', 'comment_body', 'comment_score', 'parent_id', 'comment_created_dt', 'text_raw', 'text_clean', 'keep_basic', 'is_boilerplate', 'keep_final']


In [18]:
# Compute VADER
# VADER compound score ranges from -1 to +1
#   >= 0.05  → Positive
#   <= -0.05 → Negative
#   otherwise → Neutral

if "vader_compound" not in df.columns:
    print("VADER not found → computing sentiment scores...")
    
    analyzer = SentimentIntensityAnalyzer()
    # Apply VADER to each cleaned comment
    scores = df["text_clean"].astype(str).apply(analyzer.polarity_scores)
    scores_df = pd.DataFrame(list(scores))
    
    # Rename to our consistent naming convention
    scores_df = scores_df.rename(columns={
        "compound": "vader_compound",
        "pos": "vader_pos",
        "neu": "vader_neu",
        "neg": "vader_neg"
    })
    # Merge back into main dataframe
    df = pd.concat([df.reset_index(drop=True),
                    scores_df.reset_index(drop=True)], axis=1)
else:
    print("Using existing VADER columns.")


VADER not found → computing sentiment scores...


In [19]:
print("\n=== ONE-WAY ANOVA ===")
# Test whether mean sentiment differs by company
# H0: All company means are equal
# H1: At least one company mean differs

analysis_df = df[["company", "vader_compound"]].dropna()

groups = [
    analysis_df.loc[analysis_df["company"] == c, "vader_compound"].values
    for c in analysis_df["company"].unique()
]

# Assumption check, equal variances
# Levene test:
# H0: Variances are equal across groups
lev_stat, lev_p = stats.levene(*groups)
print("Levene test p-value:", lev_p)

# ANOVA
f_stat, p_anova = stats.f_oneway(*groups)

print("F-statistic:", f_stat)
print("p-value:", p_anova)
# Save results
anova_results = pd.DataFrame({
    "F_statistic": [f_stat],
    "p_value": [p_anova],
    "levene_p_value": [lev_p]
})

anova_results.to_csv(os.path.join(TAB_DIR, "anova_results.csv"), index=False)



=== ONE-WAY ANOVA ===
Levene test p-value: 0.05439216397522384
F-statistic: 10.413373896754942
p-value: 8.473258852253513e-07


In [20]:
print("\n=== TUKEY HSD (Post-hoc) ===")

# Used only if ANOVA is significant
# Compares every pair of companies
# Controls family-wise error rate (FWER)

try:
    from statsmodels.stats.multicomp import pairwise_tukeyhsd
    
    tukey = pairwise_tukeyhsd(
        endog=analysis_df["vader_compound"],
        groups=analysis_df["company"],
        alpha=0.05
    )
    
    print(tukey)
    
    tukey_df = pd.DataFrame(
        data=tukey.summary().data[1:], 
        columns=tukey.summary().data[0]
    )
    
    tukey_df.to_csv(os.path.join(TAB_DIR, "tukey_results.csv"), index=False)

except Exception as e:
    print("Tukey skipped:", e)



=== TUKEY HSD (Post-hoc) ===
Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
   DHL  FedEx  -0.1549  0.001 -0.2521 -0.0578   True
   DHL    UPS  -0.0274  0.899   -0.13  0.0751  False
   DHL   USPS  -0.0389 0.6875 -0.1329  0.0551  False
 FedEx    UPS   0.1275  0.001  0.0506  0.2044   True
 FedEx   USPS    0.116  0.001   0.051   0.181   True
   UPS   USPS  -0.0115    0.9 -0.0844  0.0614  False
----------------------------------------------------


In [21]:
print("\n=== BINOMIAL TEST (>50% Negative) ===")

# Test whether MORE than 50% of comments are negative
#
# H0: p_negative = 0.5
# H1: p_negative > 0.5

# Convert compound score into categorical sentiment
df["sentiment_label"] = df["vader_compound"].apply(
    lambda x: "Positive" if x >= 0.05 else
              "Negative" if x <= -0.05 else
              "Neutral"
)
# Binary indicator: 1 if negative
df["neg_binary"] = (df["sentiment_label"] == "Negative").astype(int)

#1 sided binomial test
def binom_pvalue_greater(k, n, p0=0.5):
    try:
        from scipy.stats import binomtest
        return binomtest(k, n, p=p0, alternative="greater").pvalue
    except:
        from scipy.stats import binom_test
        return binom_test(k, n, p=p0, alternative="greater")

results = []

for company, sub in df.groupby("company"):
    n = len(sub)
    k = sub["neg_binary"].sum()
    pval = binom_pvalue_greater(k, n)
    
    results.append({
        "company": company,
        "n": n,
        "negative_count": k,
        "negative_proportion": k / n,
        "p_value": pval
    })

binom_df = pd.DataFrame(results)
print(binom_df)

binom_df.to_csv(os.path.join(TAB_DIR, "binomial_results.csv"), index=False)



=== BINOMIAL TEST (>50% Negative) ===
  company    n  negative_count  negative_proportion  p_value
0     DHL  205              52             0.253659      1.0
1   FedEx  597             223             0.373534      1.0
2     UPS  411             115             0.279805      1.0
3    USPS  793             232             0.292560      1.0


In [22]:
print("\n=== CHI-SQUARE TEST ===")

# Test whether sentiment distribution differs by company
#
# H0: Sentiment and company are independent
# H1: They are associated

cont_table = pd.crosstab(df["company"], df["sentiment_label"])

# Run chi-square test
chi2, p_chi2, dof, expected = chi2_contingency(cont_table)

print("Chi-square statistic:", chi2)
print("p-value:", p_chi2)
print("Degrees of freedom:", dof)

# Effect size (Cramer's V)
n_total = cont_table.values.sum()
r, c = cont_table.shape
cramers_v = np.sqrt(chi2 / (n_total * (min(r - 1, c - 1))))

chi_results = pd.DataFrame({
    "chi2": [chi2],
    "p_value": [p_chi2],
    "degrees_freedom": [dof],
    "cramers_v": [cramers_v]
})

chi_results.to_csv(os.path.join(TAB_DIR, "chi_square_results.csv"), index=False)



=== CHI-SQUARE TEST ===
Chi-square statistic: 18.217001716688817
p-value: 0.005712173619927964
Degrees of freedom: 6
